<a href="https://colab.research.google.com/github/sahil3Vedi/Enron-Email-Classifier/blob/master/ActiveTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import numpy as np
from sklearn.svm import LinearSVC
import random
import matplotlib.pyplot as plt

In [8]:
#TRAINING
ENRONFOLDER = 'drive/My Drive/Enron Spam/'
ANNO_LOC = 'drive/My Drive/Enron Spam/annotations.npy'
ANNOIND_LOC = 'drive/My Drive/Enron Spam/annotation_indices.npy'
FEATURES_LOC = 'drive/My Drive/Enron Spam/featureVectors.npy'
ENRONNOSPAMDATASET = 'drive/My Drive/Enron Spam/ham'

features = np.load(FEATURES_LOC)
ANNO = np.load(ANNOIND_LOC)
X_train = []
for each_index in ANNO:
  X_train.append(features[each_index])
Y_train = np.load(ANNO_LOC)

model1 = LinearSVC()
model1.fit(X_train, Y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [0]:
#TESTING
def fetchLabel(mailIndex):
  twoD_array = []
  feature_vector = features[mailIndex]
  twoD_array.append(feature_vector)
  file_location = os.listdir(ENRONNOSPAMDATASET)[mailIndex]
  FILENAME = os.path.join(ENRONNOSPAMDATASET, file_location)
  with open(FILENAME, 'r',encoding='utf-8',errors='ignore') as myfile:
    data = myfile.read()
    #print("Message Contents: ")
    #print(data)
    #print("Classifier Label: ")
    result = model1.predict(twoD_array)[0]
    return result

In [12]:
results = []
for x in range(1000):
  label = fetchLabel(x)
  results.append(label)
print(results.count(0))
print(results.count(1))

838
162
